**Projet 8: Traiter les images pour le système embarqué d’une voiture autonome** 

Dans ce notebook, nous allons tester différents modèles pour réaliser une segmentation en 8 catégories des images pour le système embarqué d'une voiture autonome;
Nous commencerons par des modèles "basiques", qui seront notre baseline de référence vs des modeles plus élaborés de deep learning.
Dans l'orde nous allons tester:
* 

Tous les modeles sont stockés en centralisé sur MLflow (local).
Le modèle le plus performant (avec la meilleure accuracy) sera utilisé pour le déploiement en production sur le cloud via un API de prediction.
Derriere, nous developperons également un pipeline CI/CD qui nous aidera à déployer le dernier modele automatiquement sur le cloud sur simple PUSH via Git Hub Actions;

> "View" > "Table of contents" pour voir le plan du notebook

# 1. Imports et paramétrages

## 1.1 Installation des librairies

In [1]:
!pip install pandas numpy matplotlib scikit-learn missingno mlflow tf-keras tqdm albumentations

In [2]:
# !pip install pandas numpy seaborn matplotlib scikit-learn missingno nltk 
# !pip install gensim transformers tensorflow-cpu==2.16.1 tf-keras==2.16.0 tensorflow_hub plot-keras-history
# !pip install opencv-python-headless opencv-contrib-python 

## 1.2 Import des Librairies

In [3]:
import numpy as np
import os
import tqdm
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from pathlib import Path
import albumentations as A
import cv2

In [4]:
! pip freeze

absl-py==2.1.0
albucore==0.0.23
albumentations==2.0.5
alembic==1.14.1
annotated-types==0.7.0
anyio==4.8.0
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asttokens==3.0.0
astunparse==1.6.3
async-lru==2.0.4
attrs==25.1.0
babel==2.17.0
beautifulsoup4==4.13.3
bleach==6.2.0
blinker==1.9.0
cachetools==5.5.2
certifi==2025.1.31
cffi==1.17.1
charset-normalizer==3.4.1
click==8.1.8
cloudpickle==3.1.1
colorama==0.4.6
comm==0.2.2
contourpy==1.3.1
cycler==0.12.1
databricks-sdk==0.44.1
debugpy==1.8.12
decorator==5.2.1
defusedxml==0.7.1
Deprecated==1.2.18
docker==7.1.0
executing==2.2.0
fastjsonschema==2.21.1
Flask==3.1.0
flatbuffers==25.2.10
fonttools==4.56.0
fqdn==1.5.1
gast==0.6.0
gitdb==4.0.12
GitPython==3.1.44
google-auth==2.38.0
google-pasta==0.2.0
graphene==3.4.3
graphql-core==3.2.6
graphql-relay==3.2.0
greenlet==3.1.1
grpcio==1.70.0
h11==0.14.0
h5py==3.13.0
httpcore==1.0.7
httpx==0.28.1
idna==3.10
importlib_metadata==8.5.0
ipykernel==6.29.5
ipython==9.0.0
ipython_pygments_lexers=

## 1.3 Paramétrages

In [5]:
#parametrage de panda pour afficher toutes les colonnes sans les tronquer
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Application des paramètres de style par défaut de Seaborn
sns.set()

# disable WARNING, INFO and DEBUG logging everywhere
# logging.disable(logging.WARNING) 

NameError: name 'pd' is not defined

In [ ]:
# Configuration MLflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("analyse_sentiment_tweets")

## 1.4 Fonctions

### 1.4.1 Obtention des chemins pour images et masques

In [ ]:
# Extensions valides pour les images
IMAGE_EXTENSIONS = {".png", ".jpg", ".jpeg"}

In [ ]:

def get_image_paths(root_dir):
    """
    Récupère les chemins des images en parcourant récursivement les dossiers.
    Trie les chemins en fonction de leur emplacement relatif pour assurer l'alignement entre images et masques.
    Ignore les images du dossier 'test/' car leurs masques relatifs ne sont pas fournis par Cityscape.
    """
    paths = [
        p for p in Path(root_dir).rglob("*") 
        if p.suffix.lower() in IMAGE_EXTENSIONS
        and ("train" in str(p) or "val" in str(p))  # Ignore "test/"
    ]
    return sorted(paths, key=lambda p: p.relative_to(root_dir))


In [ ]:
def get_mask_paths(root_dir, mask_type="_gtFine_labelIds"):
    """
    Récupère les chemins des masques en filtrant ceux contenant un type spécifique.
    Utilise les masques de segmentation sémantique (_gtFine_labelIds).
    Ignore les masques du dossier 'test/' car ils ne sont pas fournis.
    """
    paths = [
        p for p in Path(root_dir).rglob("*")
        if p.suffix.lower() in IMAGE_EXTENSIONS 
        and mask_type in p.stem 
        and ("train" in str(p) or "val" in str(p))  # Ignore "test/"
    ]
    return sorted(paths, key=lambda p: p.relative_to(root_dir))


### 1.4.2 Simplification des catégories

code pour visualiser les différentes categs de cityscapes. 
Ce code provient directement du github de cityscapes: 
https://github.com/mcordts/cityscapesScripts/blob/master/cityscapesscripts/helpers/labels.py

In [ ]:
from collections import namedtuple

In [ ]:
#--------------------------------------------------------------------------------
# Definitions
#--------------------------------------------------------------------------------

# a label and all meta information
Label = namedtuple( 'Label' , [

    'name'        , # The identifier of this label, e.g. 'car', 'person', ... .
                    # We use them to uniquely name a class

    'id'          , # An integer ID that is associated with this label.
                    # The IDs are used to represent the label in ground truth images
                    # An ID of -1 means that this label does not have an ID and thus
                    # is ignored when creating ground truth images (e.g. license plate).
                    # Do not modify these IDs, since exactly these IDs are expected by the
                    # evaluation server.

    'trainId'     , # Feel free to modify these IDs as suitable for your method. Then create
                    # ground truth images with train IDs, using the tools provided in the
                    # 'preparation' folder. However, make sure to validate or submit results
                    # to our evaluation server using the regular IDs above!
                    # For trainIds, multiple labels might have the same ID. Then, these labels
                    # are mapped to the same class in the ground truth images. For the inverse
                    # mapping, we use the label that is defined first in the list below.
                    # For example, mapping all void-type classes to the same ID in training,
                    # might make sense for some approaches.
                    # Max value is 255!

    'category'    , # The name of the category that this label belongs to

    'categoryId'  , # The ID of this category. Used to create ground truth images
                    # on category level.

    'hasInstances', # Whether this label distinguishes between single instances or not

    'ignoreInEval', # Whether pixels having this class as ground truth label are ignored
                    # during evaluations or not

    'color'       , # The color of this label
    ] )

labels = [
    #       name                     id    trainId   category            catId     hasInstances   ignoreInEval   color
    Label(  'unlabeled'            ,  0 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'ego vehicle'          ,  1 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'rectification border' ,  2 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'out of roi'           ,  3 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'static'               ,  4 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'dynamic'              ,  5 ,      255 , 'void'            , 0       , False        , True         , (111, 74,  0) ),
    Label(  'ground'               ,  6 ,      255 , 'void'            , 0       , False        , True         , ( 81,  0, 81) ),
    Label(  'road'                 ,  7 ,        0 , 'flat'            , 1       , False        , False        , (128, 64,128) ),
    Label(  'sidewalk'             ,  8 ,        1 , 'flat'            , 1       , False        , False        , (244, 35,232) ),
    Label(  'parking'              ,  9 ,      255 , 'flat'            , 1       , False        , True         , (250,170,160) ),
    Label(  'rail track'           , 10 ,      255 , 'flat'            , 1       , False        , True         , (230,150,140) ),
    Label(  'building'             , 11 ,        2 , 'construction'    , 2       , False        , False        , ( 70, 70, 70) ),
    Label(  'wall'                 , 12 ,        3 , 'construction'    , 2       , False        , False        , (102,102,156) ),
    Label(  'fence'                , 13 ,        4 , 'construction'    , 2       , False        , False        , (190,153,153) ),
    Label(  'guard rail'           , 14 ,      255 , 'construction'    , 2       , False        , True         , (180,165,180) ),
    Label(  'bridge'               , 15 ,      255 , 'construction'    , 2       , False        , True         , (150,100,100) ),
    Label(  'tunnel'               , 16 ,      255 , 'construction'    , 2       , False        , True         , (150,120, 90) ),
    Label(  'pole'                 , 17 ,        5 , 'object'          , 3       , False        , False        , (153,153,153) ),
    Label(  'polegroup'            , 18 ,      255 , 'object'          , 3       , False        , True         , (153,153,153) ),
    Label(  'traffic light'        , 19 ,        6 , 'object'          , 3       , False        , False        , (250,170, 30) ),
    Label(  'traffic sign'         , 20 ,        7 , 'object'          , 3       , False        , False        , (220,220,  0) ),
    Label(  'vegetation'           , 21 ,        8 , 'nature'          , 4       , False        , False        , (107,142, 35) ),
    Label(  'terrain'              , 22 ,        9 , 'nature'          , 4       , False        , False        , (152,251,152) ),
    Label(  'sky'                  , 23 ,       10 , 'sky'             , 5       , False        , False        , ( 70,130,180) ),
    Label(  'person'               , 24 ,       11 , 'human'           , 6       , True         , False        , (220, 20, 60) ),
    Label(  'rider'                , 25 ,       12 , 'human'           , 6       , True         , False        , (255,  0,  0) ),
    Label(  'car'                  , 26 ,       13 , 'vehicle'         , 7       , True         , False        , (  0,  0,142) ),
    Label(  'truck'                , 27 ,       14 , 'vehicle'         , 7       , True         , False        , (  0,  0, 70) ),
    Label(  'bus'                  , 28 ,       15 , 'vehicle'         , 7       , True         , False        , (  0, 60,100) ),
    Label(  'caravan'              , 29 ,      255 , 'vehicle'         , 7       , True         , True         , (  0,  0, 90) ),
    Label(  'trailer'              , 30 ,      255 , 'vehicle'         , 7       , True         , True         , (  0,  0,110) ),
    Label(  'train'                , 31 ,       16 , 'vehicle'         , 7       , True         , False        , (  0, 80,100) ),
    Label(  'motorcycle'           , 32 ,       17 , 'vehicle'         , 7       , True         , False        , (  0,  0,230) ),
    Label(  'bicycle'              , 33 ,       18 , 'vehicle'         , 7       , True         , False        , (119, 11, 32) ),
    Label(  'license plate'        , -1 ,       -1 , 'vehicle'         , 7       , False        , True         , (  0,  0,142) ),
]


#--------------------------------------------------------------------------------
# Main for testing
#--------------------------------------------------------------------------------

# just a dummy main
if __name__ == "__main__":
    # Print all the labels
    print("List of cityscapes labels:")
    print("")
    print("    {:>21} | {:>3} | {:>7} | {:>14} | {:>10} | {:>12} | {:>12}".format( 'name', 'id', 'trainId', 'category', 'categoryId', 'hasInstances', 'ignoreInEval', 'color' ))
    print("    " + ('-' * 98))
    for label in labels:
        print("    {:>21} | {:>3} | {:>7} | {:>14} | {:>10} | {:>12} | {:>12}".format( label.name, label.id, label.trainId, label.category, label.categoryId, label.hasInstances, label.ignoreInEval, label.color ))
    print("")

In [ ]:
# Catégories simplifiées
categs = {
    'void': [0, 1, 2, 3, 4, 5, 6],
    'flat': [7, 8, 9, 10],
    'construction': [11, 12, 13, 14, 15, 16],
    'object': [17, 18, 19, 20],
    'nature': [21, 22],
    'sky': [23],
    'human': [24, 25],
    'vehicle': [26, 27, 28, 29, 30, 31, 32, 33, -1]
}

In [ ]:
def make_mask(img):
    """
    Simplifie les labels du masque en transformant les 32 sous-categs en 8 categs principales.
    """
    mask = np.zeros((img.shape[0], img.shape[1], 8))
    for i, cat in enumerate(categs.values()):  # Parcourt les catégories
        mask[:, :, i] = np.isin(img, cat)  
    return mask.astype(np.uint8)  # Assurer uint8

### 1.4.3 Colorisation du mask

In [ ]:
# Dictionnaire de couleurs avec des clés en entiers
color_map = {
    0: [50, 50, 50],     # void (gris foncé neutre)
    1: [210, 180, 140],  # flat (ton sable/taupe)
    2: [220, 120, 60],   # construction (brique/orange)
    3: [255, 215, 0],    # object (jaune doré)
    4: [60, 180, 75],    # nature (vert dynamique)
    5: [135, 206, 235],  # sky (bleu ciel clair)
    6: [255, 0, 0],      # human (rouge vif)
    7: [0, 0, 255]       # vehicle (bleu vif)
}


In [ ]:
# # Dictionnaire de couleurs avec des clés en entiers
# color_map = {
#     0: [50, 50, 50],       # void
#     1: [128, 64,128],   # flat
#     2: [102,102,156], # construction
#     3: [153,153,153],   # object
#     4: [0, 255, 0],     # nature
#     5: [0, 204, 204],   # sky
#     6: [255, 0, 0],     # human
#     7: [0, 0, 255]      # vehicle
# }

In [ ]:
# # Dictionnaire de couleurs avec des clés en entiers
# color_map = {
#     0: [0, 0, 0],       # void
#     1: [153, 153, 0],   # flat
#     2: [255, 204, 204], # construction
#     3: [255, 0, 127],   # object
#     4: [0, 255, 0],     # nature
#     5: [0, 204, 204],   # sky
#     6: [255, 0, 0],     # human
#     7: [0, 0, 255]      # vehicle
# }

In [ ]:
def apply_color_map(mask):
    """
    Applique une colormap personnalisée à un masque en niveaux de gris.
    """
    h, w = mask.shape
    color_mask = np.zeros((h, w, 3), dtype=np.uint8)

    for label, color in color_map.items():
        color_mask[mask == label] = color  # Remplace chaque pixel par sa couleur

    return color_mask

In [ ]:
def apply_color_map(mask):
    """
    Applique une colormap personnalisée à un masque en one-hot encoding ou en indices. Pour la visualisation de la segmentation
    """
    # Si le masque est en one-hot encoding, on le convertit en indices
    if mask.ndim == 3 and mask.shape[-1] > 1:
        mask = np.argmax(mask, axis=-1) # Transformation en single-channel pour affichage (on garde la classe la plus présente par pixel)

    h, w = mask.shape
    color_mask = np.zeros((h, w, 3), dtype=np.uint8)

    for label, color in color_map.items():
        color_mask[mask == label] = color  # Remplace chaque pixel par sa couleur

    return color_mask

### 1.4.4 Process & save masks

In [ ]:
def process_and_save_masks(base_dir, mask_dir, save_dir, mask_type, mask_size):
    """
    Traite et sauvegarde les masques de segmentation sous format .npy en conservant 
    la structure des dossiers d'origine (train/val). 

    Args:
        base_dir (Path or str): Répertoire de base où enregistrer les masques convertis.
        mask_dir (Path or str): Répertoire contenant les masques originaux.
        mask_type (str, optional): Type de masque à filtrer. Default: "_gtFine_labelIds".
        mask_size (tuple, optional): Taille cible des masques. Default: (256, 256).
    """

    save_dir.mkdir(parents=True, exist_ok=True)  # Création du dossier si inexistant

    # Récupération des chemins des masques filtrés
    mask_paths = get_mask_paths(mask_dir, mask_type=mask_type)

    # Traitement des masques
    for file_path in tqdm.tqdm(mask_paths, desc="Processing masks"):
        try:
            # Déterminer le sous-dossier (train ou val)
            relative_path = file_path.relative_to(mask_dir)
            subdir = relative_path.parent  # Ex: train ou val

            # Construire le chemin de sauvegarde
            save_subdir = save_dir / subdir
            save_subdir.mkdir(parents=True, exist_ok=True)

            save_path = save_subdir / (file_path.stem + ".npy")

            # Vérifier si le fichier existe déjà
            if save_path.exists():
                continue  # Évite de traiter un fichier déjà sauvegardé

            # Vérification du format d'image
            if file_path.suffix.lower() not in IMAGE_EXTENSIONS:
                print(r" /!\  Fichier ignoré (mauvais format) : {file_path}")
                continue

            # Chargement et conversion du masque
            mask = image.load_img(file_path, color_mode="grayscale", target_size=mask_size)
            mask = np.array(mask, dtype=np.uint8)  # Convertir en tableau NumPy

            # Simplification du masque
            mask_simplified = make_mask(mask)

            # Sauvegarde du masque simplifié
            np.save(save_path, mask_simplified, allow_pickle=False)

        except Exception as e:
            print(f"X Erreur lors du traitement de {file_path} : {e}")

# 2. Data pre-processing & augmentation

In [ ]:
# Definition des chemins d'accès

In [ ]:
# Définition des chemins
base_dir = Path("C:/Users/flore/Openclassrooms/Projet 8/data/original") 

image_dir = base_dir / "images" # chemin du dossier d'images
mask_dir = base_dir / "masks" # chemin du dossier de masques


pour un entrainement plus rapide des modèles, nous allons:
* passer les masks de 32 sous categs à 8 categs principales
* enregistrer les fichiers img sous le format .npy, plus rapide à charger

## 2.1 Visualisation et simplification des catégories pour une image

In [ ]:

# Liste des images et des masques labelIds (segmentation sémantique)
image_list = get_image_paths(image_dir)
mask_list = get_mask_paths(mask_dir, mask_type="_gtFine_labelIds")


# Vérification du nombre d'images
print(f"Nombre d'images (hors Test): {len(image_list)}")
print(f"Nombre de masques (hors Test): {len(mask_list)}")


In [ ]:
# Chargement d'une image et de son masque correspondant:

idx = 2830  # on peut tester différents indices
image_path = image_list[idx]
mask_path = mask_list[idx]

# Chargement de l'image
img = image.load_img(image_path)  # sans Resize
img = image.img_to_array(img) / 255.0  # Normalisation

# Chargement du masque
mask = image.load_img(mask_path, color_mode="grayscale")
mask = np.array(mask, dtype=np.uint8)  # Convertir directement en entier pour garder les 8 labels

mask_simplified = make_mask(mask) # Simplification du masque en 8 categs au lieu de 32;
mask_colorized = apply_color_map(mask_simplified) # Colorisation


# Affichage

print(f"Chemin de l'image : {image_path.relative_to(base_dir)}")

fig, ax = plt.subplots(1, 3, figsize=(15, 10))
ax[0].imshow(img)
ax[0].set_title("Image originale")
ax[1].imshow(mask)
ax[1].set_title("Masque labelID original")
ax[2].imshow(mask_colorized)
ax[2].set_title("Masque labelID colorisé")
plt.show()

## 2.2. Simplification categs et sauvegardes des masques en .npy

Étape	           Image	    Mask
Redimensionnement	✅ OUI	    ✅ OUI
Normalisation	    ✅ /255	❌ NON
Format final	(H, W, 3), float32	(H, W, 8), one-hot encodé

In [ ]:
# save_dir = base_dir / "masks_npy"  # Dossier de sauvegarde des masques au format .npy
# os.makedirs(save_dir, exist_ok=True)  # Création si inexistant

# mask_type = "_gtFine_labelIds"
# mask_size = (256, 256)

# # Récupération des chemins des masques filtrés
# mask_paths = get_mask_paths(mask_dir, mask_type=mask_type)

# # Traitement des masques
# for file_path in tqdm.tqdm(mask_paths):
#     try:
#         # Déterminer le sous-dossier (train ou val)
#         relative_path = file_path.relative_to(mask_dir)
#         subdir = relative_path.parent  # Récupérer le sous-dossier (ex: train ou val)

#         # Construire le chemin de sauvegarde en répliquant la structure
#         save_subdir = save_dir / subdir
#         save_subdir.mkdir(parents=True, exist_ok=True)  # Création récursive du dossier

#         # Création du chemin de sauvegarde
#         save_path = save_subdir / (file_path.stem + ".npy")

#         # Vérifier si le fichier existe déjà
#         if save_path.exists():
#             continue  # Passer au fichier suivant

#         # Vérification du format d'image avant chargement
#         if file_path.suffix.lower() not in IMAGE_EXTENSIONS:
#             print(f" /!\  Fichier ignoré (mauvais format) : {file_path}")
#             continue

#         # Chargement du masque
#         mask = image.load_img(file_path,  color_mode="grayscale", target_size=mask_size)
#         mask = np.array(mask, dtype=np.uint8)  # Convertir directement en entier

#         # Simplification du masque
#         mask_simplified = make_mask(mask)

#         # Création du chemin de sauvegarde
#         save_path = save_subdir / (file_path.stem + ".npy")

#         # Sauvegarde du masque simplifié
#         np.save(save_path, mask_simplified)

#     except Exception as e:
#         print(f"X Erreur lors du traitement de {file_path} : {e}")


In [ ]:
save_dir = Path(base_dir) /  "masks_npy"
mask_type = "_gtFine_labelIds"
mask_size = (256, 256)

process_and_save_masks(base_dir, mask_dir, save_dir, mask_type, mask_size)

## 2.3. Data augmentation

In [ ]:
# Définition des transformations pour l'augmentation
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=15, p=0.5),
    A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.3),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.3),
    A.GaussNoise(var_limit=(10, 50), p=0.2),
    A.CoarseDropout(max_holes=5, max_height=50, max_width=50, p=0.3),
], additional_targets={'mask': 'mask'})

In [ ]:
# Dossiers source & destination
base_dir = Path("C:/Users/flore/Openclassrooms/Projet 8/data")
image_dir = base_dir / "original" / "images"
mask_dir = base_dir / "original" / "masks"
augmented_dir = base_dir / "augmented"

In [ ]:
# paramétrages:
mask_type = "_gtFine_labelIds"
target_size = (256, 256)

Applique une augmentation aux images et leurs masques, enregistre l'image augmentée en PNG et le masque en .npy.
Resize (256,256) APRES l'augmentation pour éviter la perte d'information. resize permet de passer au modele des images légères

In [ ]:
# Liste des images
image_paths = get_image_paths(image_dir)

for image_path in tqdm.tqdm(image_paths, desc="Processing images & masks"):
    try:
        # Déterminer le sous-dossier (train ou val)
        relative_path = image_path.relative_to(image_dir)
        subdir = relative_path.parent  # Ex: train ou val

        # Construire les chemins de sauvegarde
        save_image_subdir = augmented_dir / "images" / subdir
        save_mask_subdir = augmented_dir / "masks_npy" / subdir  # Stockage des masques en .npy ici

        # Création des dossiers parents si nécessaire
        save_image_subdir.mkdir(parents=True, exist_ok=True)
        save_mask_subdir.mkdir(parents=True, exist_ok=True)

        # Définition des chemins finaux des fichiers sauvegardés
        save_image_path = save_image_subdir / f"{image_path.stem}.png"
        save_mask_path = save_mask_subdir / f"{image_path.stem}.npy"

        # Vérification si le masque existe bien pour l'image donnée
        mask_relative_path = image_path.relative_to(image_dir)  # Récupération du chemin relatif
        mask_name = image_path.stem.replace("_leftImg8bit", "") # récupération du nom de base sans `_leftImg8bit`
        mask_path = mask_dir / mask_relative_path.parent / f"{mask_name}_gtFine_labelIds.png"

        if not mask_path.exists():
            print(f"Masque non trouvé pour {image_path.name}, passage...")
            continue

        # Chargement de l'image et du masque sans redimensionnement
        img = image.load_img(image_path)  # Chargement en RGB, taille originale
        mask = image.load_img(mask_path, color_mode="grayscale")  # Chargement en niveaux de gris

        img = np.array(img)
        mask = np.array(mask, dtype=np.uint8)  # Convertir en tableau NumPy

        # Vérification que tout est OK
        if img is None or mask is None:
            print(f"Problème de chargement pour {image_path.name}, passage...")
            continue

        # Appliquer l'augmentation aux images et aux masques
        augmentation = transform(image=img, mask=mask)
        image_aug, mask_aug = augmentation["image"], augmentation["mask"]

        # Resize après augmentation
        image_aug = cv2.resize(image_aug, target_size, interpolation=cv2.INTER_LINEAR)
        mask_aug = cv2.resize(mask_aug, target_size, interpolation=cv2.INTER_NEAREST)

        # Simplification des catégories du masque (ex: de 34 à 8 classes)
        mask_simplified = make_mask(mask_aug)

        # Sauvegarde des fichiers
        cv2.imwrite(str(save_image_path), image_aug)
        np.save(save_mask_path, mask_simplified, allow_pickle=False)

    except Exception as e:
        print(f"Erreur sur {image_path.name} : {e}")

print("Augmentation terminée !")

# 3. DataGenerator

le DataGenerator est un script python sauvegardé dans le meme dossier que ce notebook

In [ ]:
from DataGenerator import DataGenerator

# paramétrage
batch_size = 32
img_size=(256, 256)

# chemins d'accès (uniquement avec les images d'origines):
base_dir = Path("C:/Users/flore/Openclassrooms/Projet 8/data/original")
train_image_dir = base_dir / "images" / "train"
train_mask_dir = base_dir / "masks_npy"/ "train"
val_image_dir = base_dir / "images" / "val"
val_mask_dir = base_dir / "masks_npy"/ "val"


train_image_paths = get_image_paths (train_image_dir)
train_mask_paths = get_mask_paths (train_mask_dir)
val_image_paths = get_image_paths (val_image_dir)
val_mask_paths = get_mask_paths (val_mask_dir)

In [ ]:
# Instanciation du générateur

train_generator = DataGenerator(
    image_paths=train_image_paths,
    mask_paths=train_mask_paths,
    batch_size=batch_size,
    img_size=img_size,
    num_classes=8,
    augmentation=transform,
    one_hot=True,
    shuffle=True
)

val_generator = DataGenerator(
    image_paths=val_image_paths,
    mask_paths=val_mask_paths,
    batch_size=batch_size,
    img_size=img_size,
    num_classes=8,
    one_hot=True,
    shuffle=False
)

In [ ]:
#Test du data generator
# Récupérer un batch
X, Y = train_generator[0]

# Vérifier les formes des données
print("Shape des images :", X.shape)  # Doit être (batch_size, img_size, img_size, 3)
print("Shape des masques :", Y.shape)  # Si one-hot activé : (batch_size, img_size, img_size, num_classes)

In [ ]:
a

### 4.1 Regression log avec CountVectorizer

CountVectorizer	convertit le texte en une matrice d'occurrences de mots.	
* **Avantages**: Simple, rapide à exécuter, efficace pour des vocabulaires réduits.	
* **Inconvénients**: Ne capture pas la sémantique ou l'ordre des mots ; sensible à la taille du vocabulaire.

Definition du modèle: une regression logistique, sans hyper-parmètres de départ. Ils seront ensuite affinés via l'optimisation des hyperparamètres (grid_searchCV)

In [ ]:
with mlflow.start_run(run_name="Log Reg with CountVectorizer"):

    mlflow.set_tag("Approach", "CountVectorizer")

    #----------------------------
    # Hyperparamètres
    params = {
        "C": 1, #[0.01, 0.1, 1, 10, 100]
        "solver": 'lbfgs', #'newton-cg','lbfgs','liblinear', 'saga']
        "penalty": 'l2', #['l1', 'l2'], 
        "random_state": 42,
        "n_jobs": -1,
    }
    mlflow.log_params(params) #log des paramètres

    #----------------------------
    # Modèle et pipeline
    model = LogisticRegression(**params)
    pipeline = make_pipeline(
        CountVectorizer(stop_words='english', max_df=0.95, min_df=2), # Etape 1: transformation des features en vecteur
        model                                                         # Étape 2 : Modèle de regression logistique 
    )
    # pipeline = Pipeline([
    #     ('vectorizer', CountVectorizer(stop_words='english', max_df=0.95, min_df=2)), # Etape 1: transformation des features en vecteur
    #     ('model', model)                                      # Étape 2 : Modèle de regression logistique
    # ])
    
    # Log du pipeline
    mlflow.sklearn.log_model(pipeline, "LogReg_CountVectorizer")

    
    #----------------------------
    # Calcul de l'accuracy, sans rechercher le seuil optimal
    cv_scores_pipeline = cross_val_score(pipeline, data['sentence_bow_lem'], data['target'], cv=5, scoring='accuracy')
    # mlflow.log_metric("Accuracy sans seuil optimal", round(np.mean(cv_scores_pipeline),3))

    #----------------------------
    # Prédiction et métrics en recherchant le seuil optimal
    # Utilisation de 'predict_proba' pour obtenir les probabilités
    y_proba = cross_val_predict(pipeline, data['sentence_bow_lem'], data['target'], cv=5, method='predict_proba')
    results, conf_matrix = model_metrics(data['target'], y_proba)
    
    #logs des résultats dans MLflow:
    mlflow.log_metric("Accuracy", results['Accuracy'][-1])
    mlflow.log_metric("F1_Score", results['F1-score'][-1])
    mlflow.log_metric("Coût métier", results['Coût métier'][-1])
    mlflow.log_metric("FN", results['FN'][-1])
    mlflow.log_metric("FP", results['FP'][-1])

    #----------------------------
    # Matrice de confusion
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title("Matrice de confusion avec seuil optimal")
    plt.xlabel("Prédictions")
    plt.ylabel("Vrais labels")
    
    # Enregistrement avant affichage
    plt.savefig("conf_matrix.png")
    mlflow.log_artifact("conf_matrix.png")
    
    plt.show()
    
        

### 4.2  Regression log avec TF-IDF

TF-IDF = Term Frequency - Inverse Document Frequency
On pondère l'importance d'un mot dans un document, par rapport à l'ensemble du corpus (ici tous nos articles vendus sur le site e-com)

* **Avantages:** Meilleure différenciation entre les termes fréquents et rares, réduit l'impact des mots peu informatifs.
* **Inconvénients:** là encore, insensible à l'ordre et au contexte des mots ; dépend du corpus pour une bonne pondération.

In [ ]:
with mlflow.start_run(run_name="Log Reg with TF-IDF"):

    mlflow.set_tag("Approach", "TF-IDF")

    #----------------------------
    # Hyperparamètres
    params = {
        "C": 1, #[0.01, 0.1, 1, 10, 100]
        "solver": 'lbfgs', #'newton-cg','lbfgs','liblinear', 'saga']
        "penalty": 'l2', #['l1', 'l2'], 
        "random_state": 42,
        "n_jobs": -1,
    }
    mlflow.log_params(params) #log des paramètres

    #----------------------------
    # Modèle et pipeline
    model = LogisticRegression(**params)
    pipeline = make_pipeline(
        TfidfVectorizer(stop_words='english', max_df=0.95, min_df=2), # Etape 1: transformation des features en vecteur
        model                                                         # Étape 2 : Modèle de regression logistique 
    )
    # Log du pipeline
    mlflow.sklearn.log_model(pipeline, "LogReg_TF_IDF")


    
    #----------------------------
    # Calcul de l'accuracy, sans rechercher le seuil optimal
    cv_scores_pipeline = cross_val_score(pipeline, data['sentence_bow_lem'], data['target'], cv=5, scoring='accuracy')
    # mlflow.log_metric("Accuracy sans seuil optimal", round(np.mean(cv_scores_pipeline),3))

    #----------------------------
    # Prédiction et métrics en recherchant le seuil optimal
    # Utilisation de 'predict_proba' pour obtenir les probabilités
    y_proba = cross_val_predict(pipeline, data['sentence_bow_lem'], data['target'], cv=5, method='predict_proba')
    results, conf_matrix = model_metrics(data['target'], y_proba)
    
    #logs des résultats dans MLflow:
    mlflow.log_metric("Accuracy", results['Accuracy'][-1])
    mlflow.log_metric("F1_Score", results['F1-score'][-1])
    mlflow.log_metric("Coût métier", results['Coût métier'][-1])
    mlflow.log_metric("FN", results['FN'][-1])
    mlflow.log_metric("FP", results['FP'][-1])

    #----------------------------
    # Matrice de confusion
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title("Matrice de confusion avec seuil optimal")
    plt.xlabel("Prédictions")
    plt.ylabel("Vrais labels")
    
    # Enregistrement avant affichage
    plt.savefig("conf_matrix.png")
    mlflow.log_artifact("conf_matrix.png")
    
    plt.show()
    
        

# 5. Approche “Modèle sur mesure avancé”

Pour l'approche "modèle sur mesure avancé, nous allons utiliser différents réseaux profonds de neurones classique (DNN ou deep neural network) pour classifier les sentiments des tweets de facon binaire (positif ou negatif).
Nous allons tester 2 techniques d'embeddings, pour obtenir des vecteurs qui caractérisent chacun des tweets:
* Word2Vec
* FastText
Après le test de ces différents modèles couplés à Word2Vec et FastText, nous finirons en testant la performance de BERT.

## 5.1 DNN avec Word2Vec

Word2Vec apprend des représentations vectorielles denses pour les mots en utilisant des architectures de réseau neuronal (CBOW ou Skip-gram). 
Les mots proches dans l’espace vectoriel partagent des similarités contextuelles. 

* **Avantages:** Capture les relations sémantiques entre les mots, représentations continues adaptées aux modèles ML.
* **Inconvénients:** Nécessite un grand corpus pour des embeddings de qualité ; insensible aux structures complexes.

**DNN (Deep Neural Network)**

* Type : Réseau de neurones dense (Fully Connected)
  
* Caractéristiques :
Réseau simple avec des couches Dense (MLP - Multi-Layer Perceptron)
Relu pour l'activation des couches cachées, sigmoid pour la sortie (binaire)
Dropout et régularisation pour éviter l'overfitting

* C'est un empilement de couches de neurones connectés entre eux. Chaque neurone reçoit des nombres (features d'entrée), les transforme avec des calculs (poids + biais), applique une fonction d’activation (ex: ReLU) et envoie le résultat à la couche suivante.



In [ ]:
with mlflow.start_run(run_name="DNN with Word2Vec"):
    mlflow.set_tag("Approach", "Word2Vec + DNN")
    
    #-------------------------------------------------
    # 1. Paramétrage du modèle
    #-------------------------------------------------
    params = {
        # Word2Vec
        "vector_size": 100,     # Dimension des vecteurs
        "window": 5,     # Taille de la fenêtre contextuelle
        "min_count": 1,  # Ignore les mots rares
        "epochs": 50,   # Nombre d'itérations d'entraînement sur les données
        # DNN
        "dropout_rate": 0.3, #[None, 0.3, 0.5]
        "regularization": l2(0.01), #[None, l1(0.01), l2(0.01)] 
    }
    mlflow.log_params(params)

    #-------------------------------------------------
    # 2. Création du modèle Word2Vec sur l'ensemble du corpus
    #-------------------------------------------------
    
    tokenized_tweets = [text.split() for text in data['sentence_bow_lem']]
    
    w2v_model = Word2Vec(
        sentences=tokenized_tweets,
        vector_size=params["vector_size"],  
        window=params["window"],
        min_count=params["min_count"],
        epochs=params["epochs"],
        workers=10
    )

    # Sauvegarde du modèle Word2Vec dans MLflow
    w2v_model_path = "word2vec.model"
    w2v_model.save(w2v_model_path)
    mlflow.log_artifact(w2v_model_path, artifact_path="word2vec")
    
    #-------------------------------------------------
    # 3. Obtention des vecteurs pour chaque tweet
    #-------------------------------------------------
    
    def get_sentence_vector(sentence, model, dim):
        vectors = [model.wv[word] for word in sentence if word in model.wv]
        return np.mean(vectors, axis=0) if vectors else np.zeros(dim)
    
    X = np.array([get_sentence_vector(text, w2v_model, params["vector_size"]) for text in tokenized_tweets])
    y = data['target'].reset_index(drop=True)

    #-------------------------------------------------
    # 4. Split en train/test (85/15)
    #-------------------------------------------------
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
    
    #-------------------------------------------------
    # 5. Construction du modèle DNN
    #-------------------------------------------------
    
    # Modèle DNN
    model = Sequential([
        Input(shape=(params["vector_size"],)),
        Dense(64, activation='relu', kernel_regularizer=params["regularization"]),
        Dropout(params["dropout_rate"]),
        Dense(32, activation='relu', kernel_regularizer=params["regularization"]),
        Dropout(params["dropout_rate"]),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=TfAdam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    
    # Early Stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    #-------------------------------------------------
    # 6. Entraînement du modèle & Prédiction
    #-------------------------------------------------
    
    history = model.fit(
        X_train, y_train,
        epochs=params['epochs'],
        batch_size=16,
        validation_split=0.15,
        #verbose=0,
        callbacks=[early_stopping]
    )
    
    y_pred = (model.predict(X_test) > 0.5).astype(int)

    #-------------------------------------------------
    # 7. Résultats 
    #-------------------------------------------------

    show_history(history)
    plot_history(history, path=f"training_history.png")
    plt.close()
    
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = conf_matrix.ravel()
    cout_metier = fp + fn

    print("Accuracy:", accuracy)
    print("F1 Score:", f1)
    print("Matrice de confusion:\n", conf_matrix)
    print("Coût métier:", cout_metier)
    
    mlflow.log_metric("Accuracy", round(accuracy, 3))
    mlflow.log_metric("F1_Score", round(f1, 3))
    mlflow.log_metric("Coût métier", cout_metier)
    mlflow.log_metric("FN", fn)
    mlflow.log_metric("FP", fp)
    
    signature = infer_signature(X_train, model.predict(X_train))
    mlflow.keras.log_model(model, "DNN_Model", signature=signature)


## 5.2 DNN avec FastText

FastText est une technique d'embedding qui apprend des représentations vectorielles denses pour les mots en utilisant des architectures de réseau neuronal (CBOW ou Skip-gram). Contrairement à Word2Vec, il représente chaque mot comme une somme de vecteurs de sous-mots (n-grammes), ce qui permet une meilleure généralisation sur les mots rares ou inconnus. Les mots proches dans l’espace vectoriel partagent des similarités contextuelles et morphologiques.  

* **Avantages:** Capture les relations sémantiques et morphologiques entre les mots, gère mieux les mots rares ou inconnus, et améliore la robustesse aux fautes d’orthographe.  
* **Inconvénients:** Plus gourmand en mémoire et en calcul que Word2Vec ; nécessite un tuning des n-grammes pour de meilleurs résultats.

In [ ]:
with mlflow.start_run(run_name="DNN with FastText"):
    
    mlflow.set_tag("Approach", "FastText + DNN")
    
    #-------------------------------------------------
    # 1. Paramétrage du modèle
    #-------------------------------------------------
    
    params = {
        #FastText
        "vector_size": 100,     # Dimension des vecteurs
        "window": 5,     # Taille de la fenêtre contextuelle
        "min_count": 1,  # Ignore les mots rares
        "epochs": 100,   # Nombre d'itérations d'entraînement sur les données
        # DNN
        "dropout_rate": 0.3, #[None, 0.3, 0.5]
        "regularization": l2(0.01), #[None, l1(0.01), l2(0.01)] 
    }

    
    mlflow.log_params(params)
    
    # obtention des tweets tokenisés:
    tokenized_tweets = [text.split() for text in data['sentence_bow_lem']]
    
    #-------------------------------------------------
    # 2. Création du modèle FastText sur l'ensemble du corpus
    #-------------------------------------------------
    
    ft_model = FastText(
        sentences=tokenized_tweets,
        vector_size=params["vector_size"],  
        window=params["window"],
        min_count=params["min_count"],
        epochs=params["epochs"],
        workers=10
    )

    # Sauvegarde du modèle FastText dans MLflow an tant qu'artifact:
    ft_model_path = "fasttext.model"
    ft_model.save(ft_model_path)
    mlflow.log_artifact(ft_model_path, artifact_path="FastText")
    
    #-------------------------------------------------
    # 3. Obtention des vecteurs pour chaque tweet
    #-------------------------------------------------
    
    def get_sentence_vector(sentence, model, dim):
        vectors = [model.wv[word] for word in sentence if word in model.wv]
        return np.mean(vectors, axis=0) if vectors else np.zeros(dim)

    
    X = np.array([get_sentence_vector(text, ft_model, params["vector_size"]) for text in tokenized_tweets])
    y = data['target'].reset_index(drop=True)
    
    #-------------------------------------------------
    # 4. Split en train/test (85/15)
    #-------------------------------------------------
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
    
    # -------------------------------------------------
    # 5. Construction du modèle DNN
    # -------------------------------------------------
    model = Sequential([
        Input(shape=(params["vector_size"],)),
        Dense(64, activation='relu', kernel_regularizer=params["regularization"]),
        Dropout(params["dropout_rate"]),
        Dense(32, activation='relu', kernel_regularizer=params["regularization"]),
        Dropout(params["dropout_rate"]),
        Dense(1, activation='sigmoid')
    ])
        
    # model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.compile(optimizer=TfAdam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    # Early Stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    #-------------------------------------------------
    # 6. Entraînement du modèle & Prédiction
    #-------------------------------------------------
    history = model.fit(
        X_train, y_train,
        epochs=params['epochs'],
        batch_size=16,
        validation_split=0.15,
        #verbose=0,
        callbacks=[early_stopping]
    )
    
    y_pred = (model.predict(X_test) > 0.5).astype(int)

    #-------------------------------------------------
    # 7. Résultats 
    #-------------------------------------------------

    show_history(history)
    plot_history(history, path=f"training_history.png")
    plt.close()
    
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    tn, fp, fn, tp = conf_matrix.ravel()
    cout_metier = fp + fn
    
    print("Accuracy:", accuracy)
    print("F1 Score:", f1)
    print("Matrice de confusion:\n", conf_matrix)
    print("Coût métier:", cout_metier)
    
    # Logs des résultats dans MLflow
    mlflow.log_metric("Accuracy", round(accuracy, 3))
    mlflow.log_metric("F1_Score", round(f1, 3))
    mlflow.log_metric("Coût métier", cout_metier)
    mlflow.log_metric("FN", fn)
    mlflow.log_metric("FP", fp)
    
    # Enregistrement du modèle DNN dans MLflow
    signature = infer_signature(X_train, model.predict(X_train))
    mlflow.keras.log_model(model, "DNN_Model", signature=signature)


## 5.3 CNN avec FastText

**CNN (Convolutional Neural Network)**

* Type : Réseau de neurones convolutionnel (1D)
  
* Caractéristiques : Conv1D applique des filtres pour capturer des patterns locaux dans les séquences
MaxPooling réduit la dimension tout en conservant les informations importantes
Flatten transforme les sorties convolutives en un vecteur pour la classification
Relu pour l'activation, sigmoid pour la sortie binaire.

* Au lieu de tout connecter comme un DNN, un CNN applique des filtres (Conv1D) qui glissent sur l’entrée (ex: un texte). Ces filtres repèrent des motifs récurrents comme des mots-clés ou des groupes de mots. Ensuite, le max-pooling réduit la taille tout en gardant l’essentiel.

In [ ]:
with mlflow.start_run(run_name="CNN with FastText"):
    
    mlflow.set_tag("Approach", "FastText + CNN")
    
    #-------------------------------------------------
    # 1. Paramétrage du modèle
    #-------------------------------------------------
    
    params = {
        #FastText
        "vector_size": 100,     # Dimension des vecteurs
        "window": 5,     # Taille de la fenêtre contextuelle
        "min_count": 1,  # Ignore les mots rares
        "epochs": 100,   # Nombre d'itérations d'entraînement sur les données
        # CNN
        "dropout_rate": 0.3, #[None, 0.3, 0.5]
        "regularization": l2(0.01), #[None, l1(0.01), l2(0.01)] 
    }
    
    mlflow.log_params(params)
    
    # obtention des tweets tokenisés:
    tokenized_tweets = [text.split() for text in data['sentence_bow_lem']]
    
    #-------------------------------------------------
    # 2. Création du modèle FastText sur l'ensemble du corpus
    #-------------------------------------------------
    
    ft_model = FastText(
        sentences=tokenized_tweets,
        vector_size=params["vector_size"],  
        window=params["window"],
        min_count=params["min_count"],
        epochs=params["epochs"],
        workers=10
    )
    
    # Sauvegarde du modèle FastText dans MLflow an tant qu'artifact:
    ft_model_path = "fasttext.model"
    ft_model.save(ft_model_path)
    mlflow.log_artifact(ft_model_path, artifact_path="FastText")
    
    #-------------------------------------------------
    # 3. Obtention des vecteurs pour chaque tweet
    #-------------------------------------------------
    
    def get_sentence_vector(sentence, model, dim):
        vectors = [model.wv[word] for word in sentence if word in model.wv]
        return np.mean(vectors, axis=0) if vectors else np.zeros(dim)
    
    X = np.array([get_sentence_vector(text, ft_model, params["vector_size"]) for text in tokenized_tweets])
    y = data['target'].reset_index(drop=True)
    
    #-------------------------------------------------
    # 4. Split en train/test (85/15)
    #-------------------------------------------------
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
    
    
    # -------------------------------------------------
    # 5. Construction du modèle CNN
    # -------------------------------------------------
    model = Sequential([
        Input(shape=(params["vector_size"], 1)),  # Ajout d'une dimension pour les filtres CNN
        Conv1D(64, 5, activation='relu', kernel_regularizer=params["regularization"]),
        MaxPooling1D(pool_size=2),
        Dropout(params["dropout_rate"]),
        Conv1D(32, 5, activation='relu', kernel_regularizer=params["regularization"]),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    # model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.compile(optimizer=TfAdam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    # Early Stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    #-------------------------------------------------
    # 6. Entraînement du modèle & Prédiction
    #-------------------------------------------------
    history = model.fit(
        X_train, y_train,
        epochs=params['epochs'],
        batch_size=16,
        validation_split=0.15,
        #verbose=0,
        callbacks=[early_stopping]
    )

    y_pred = (model.predict(X_test) > 0.5).astype(int)

    #-------------------------------------------------
    # 7. Résultats 
    #-------------------------------------------------

    show_history(history)
    plot_history(history, path=f"training_history.png")
    plt.close()
    
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    tn, fp, fn, tp = conf_matrix.ravel()
    cout_metier = fp + fn
    
    print("Accuracy:", accuracy)
    print("F1 Score:", f1)
    print("Matrice de confusion:\n", conf_matrix)
    print("Coût métier:", cout_metier)
    
    # Logs des résultats dans MLflow
    mlflow.log_metric("Accuracy", round(accuracy, 3))
    mlflow.log_metric("F1_Score", round(f1, 3))
    mlflow.log_metric("Coût métier", cout_metier)
    mlflow.log_metric("FN", fn)
    mlflow.log_metric("FP", fp)
    
    # Enregistrement du modèle CNN dans MLflow
    signature = infer_signature(X_train, model.predict(X_train))
    mlflow.keras.log_model(model, "CNN_Model", signature=signature)

## 5.3 BiLSTM avec FastText

**BiLSTM (Bidirectional LSTM)**

* Type : Réseau de neurones récurrent (RNN) avec Long Short-Term Memory (LSTM)

* Caractéristiques :
Bidirectionnel (BiLSTM) → Lit la séquence dans les deux sens (avant et arrière)
Meilleur que CNN pour les dépendances longues
Dropout + Dense pour éviter l'overfitting
Sigmoid pour la sortie binaire

* C’est un réseau séquentiel qui lit les données dans les deux sens (gauche → droite et droite → gauche). Il stocke une mémoire à long terme pour comprendre le contexte d’un texte. Contrairement au CNN, il sait que "chat noir" et "noir chat" ne sont pas les mêmes choses.

In [ ]:
with mlflow.start_run(run_name="LSTM with FastText & CNN"):
    
    mlflow.set_tag("Approach", "FastText + CNN + BiLSTM")
    
    
#-------------------------------------------------
# 1. Paramétrage du modèle
#-------------------------------------------------


    params = {
        # FastText
        "vector_size": 100,     # Taille des vecteurs
        "window": 5,            # Fenêtre contextuelle
        "min_count": 1,         # Ignore les mots rares
        "epochs": 50,           # Nombre d'epochs pour FastText
        # LSTM
        "batch_size": 32,       # Taille des batches pour LSTM
        "lstm_units": 128,       # Nombre de neurones LSTM (BiLSTM donc 2x)
        "dropout_rate": 0.5,    # Taux de Dropout
        "conv_filters": 64,     # Filtres pour la couche Conv1D
        "conv_kernel_size": 5   # Taille du noyau Conv1D
    }
    
    mlflow.log_params(params)
    
    # Obtention des tweets tokenisés
    tokenized_tweets = [text.split() for text in data['sentence_bow_lem']]
    
    #-------------------------------------------------
    # 2. Création du modèle FastText sur l'ensemble du corpus
    #-------------------------------------------------
    
    ft_model = FastText(
        sentences=tokenized_tweets,
        vector_size=params["vector_size"],  
        window=params["window"],
        min_count=params["min_count"],
        epochs=params["epochs"],
        workers=10
    )
    
    # Sauvegarde du modèle FastText dans MLflow en tant qu'artifact
    ft_model_path = "fasttext.model"
    ft_model.save(ft_model_path)
    mlflow.log_artifact(ft_model_path, artifact_path="FastText")
    
    #-------------------------------------------------
    # 3. Obtention des vecteurs pour chaque tweet
    #-------------------------------------------------
    

    def get_sentence_vector(sentence, model, dim):
        vectors = [model.wv[word] for word in sentence if word in model.wv]
        if vectors:
            return np.concatenate([np.mean(vectors, axis=0), np.max(vectors, axis=0), np.min(vectors, axis=0)])
        else:
            return np.zeros(dim * 3)
    
    X = np.array([get_sentence_vector(text, ft_model, params["vector_size"]) for text in tokenized_tweets])
    y = data['target'].reset_index(drop=True).values  # Convertir en array
    
    #-------------------------------------------------
    # 4. Split en train/test (85/15)
    #-------------------------------------------------
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
    
    # -------------------------------------------------
    # 5. Construction du modèle CNN + BiLSTM
    # -------------------------------------------------
    
    model = Sequential([
        Input(shape=(params["vector_size"], 1)),  # Format (timesteps, features)
        Bidirectional(LSTM(128, dropout=0.1, recurrent_dropout=0.1)),  # BiLSTM 
        Dense(128, activation='relu'),     # Dense avec réduction des unités et un dropout plus bas
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])

    
    model.compile(optimizer=TfAdam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    
    # Early Stopping & Model Checkpoint
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    checkpoint = ModelCheckpoint("best_model.h5", monitor='val_loss', save_best_only=True, mode='min')
    
    #-------------------------------------------------
    # 6. Entraînement du modèle & Prédiction
    #-------------------------------------------------
    history = model.fit(
        X_train, y_train,
        epochs=params['epochs'],
        batch_size=params["batch_size"],
        validation_split=0.15,
        callbacks=[early_stopping, checkpoint]
    )
    
    # Chargement du meilleur modèle
    model.load_weights("best_model.h5")
    
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    
    #-------------------------------------------------
    # 7. Résultats 
    #-------------------------------------------------
    
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    tn, fp, fn, tp = conf_matrix.ravel()
    cout_metier = fp + fn
    
    print("Accuracy:", accuracy)
    print("F1 Score:", f1)
    print("Matrice de confusion:\n", conf_matrix)
    print("Coût métier:", cout_metier)
    
    # Logs des résultats dans MLflow
    mlflow.log_metric("Accuracy", round(accuracy, 3))
    mlflow.log_metric("F1_Score", round(f1, 3))
    mlflow.log_metric("Coût métier", cout_metier)
    mlflow.log_metric("FN", fn)
    mlflow.log_metric("FP", fp)
    
    # Enregistrement du modèle LSTM dans MLflow
    signature = infer_signature(X_train, model.predict(X_train))
    mlflow.keras.log_model(model, "LSTM_Model", signature=signature)


En conclusion, nous voyons que tous ces modèles, même si ils ne sont pas totalement optimisés, sont plus lents et donnent de bien moins bons résultats que notre baseline, une simple régression logistique...
Passons maintenant à BERT.

## 5.3 BERT

BERT Hugging face:
BERT (Bidirectional Encoder Representations from Transformers) est un modèle de langage pré-entraîné utilisant l'architecture Transformer. Contrairement aux modèles traditionnels, il est bidirectionnel, ce qui lui permet de mieux capturer le contexte des mots dans une phrase. BERT est particulièrement puissant pour des tâches de compréhension du langage comme la classification de texte, la réponse à des questions et l'extraction d'entités.
On utilisera ici un tokenizer propre à BERT.

In [ ]:
print(multiprocessing.cpu_count())  # Nombre total de cœurs logiques

In [ ]:
print(tf.__version__)
print("GPU disponible :", tf.config.list_physical_devices('GPU'))
print(tf.test.is_built_with_cuda())

Voici les étapes principales du code ci-dessous:

1. Initialisation du modèle et du tokenizer
On charge DistilBERT ainsi que son tokenizer qui convertira le texte en données exploitables par le modèle.

2. Paramètres d'entraînement
On définit les réglages du modèle : nombre d'époques (combien de fois on passe sur les données), taille des lots de données traités en même temps, et le taux d'apprentissage (vitesse d'adaptation du modèle).

3. Préparation des données, divisées en trois groupes :

* 80% pour l'entraînement
* 10% pour la validation (vérifier que le modèle ne s'entraîne pas trop sur un même schéma)
* 10% pour le test final
Ensuite, on transforme ces textes en tokens exploitables par le modèle.

4. Création des datasets TensorFlow
On met en forme les données pour qu'elles soient prêtes à être utilisées par TensorFlow, en les regroupant en lots et en optimisant leur chargement.

5. Définition de l'optimiseur et compilation
On choisit l'algorithme qui ajustera le modèle (AdamWeightDecay) et la fonction qui mesurera l'erreur (loss function).

6. Entraînement du modèle
Le modèle apprend en analysant les données d’entraînement et en ajustant ses paramètres. On vérifie la progression en regardant les performances sur le jeu de validation.

7. Sauvegarde et log des résultats
On sauvegarde le modèle et le tokenizer, et on enregistre les résultats dans MLflow pour pouvoir les analyser plus tard.

In [ ]:
# import os
# import mlflow
# import mlflow.tensorflow
# import tensorflow as tf
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
# from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from transformers import AdamWeightDecay

# XLA (Accelerated Linear Algebra) optimise les opérations TensorFlow en fusionnant plusieurs calculs.
os.environ["TF_XLA_FLAGS"] = "--tf_xla_auto_jit=2"
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "1"

# Démarrage de la session MLflow
with mlflow.start_run(run_name="DistilBERT_Model_Classification"):
    mlflow.set_tag("Approach", "DistilBERT")
    
    #-------------------------------------------------
    # 1. Initialisation du modèle et du tokenizer
    #-------------------------------------------------
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    model = TFDistilBertForSequenceClassification.from_pretrained(
        'distilbert-base-uncased', 
        num_labels=2, # classification binaire => num_labels=2
        from_pt=True # Gain de temps : évite la conversion automatique PyTorch → TensorFlow.
    )

    #-------------------------------------------------
    # 2. Paramètres d'entraînement
    #-------------------------------------------------
    params = {
        "epochs": 3,  
        "batch_size": 32,  
        "lr": 2e-5,  
    }

    # Log des paramètres dans MLflow
    mlflow.log_params(params)
    
    #-------------------------------------------------
    # 3. Tokenisation et split des données (80% train, 10% validation, 10% test)
    #-------------------------------------------------
    X = [str(tweet) for tweet in data['sentence_BERT']]
    y = data['target'].reset_index(drop=True)

    #Double train_test_split pour definir un dataset de validation:
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

    # Tokenisation des ensembles
    train_tokens = tokenizer(X_train, padding=True, truncation=True, return_tensors="tf", max_length=64)  
    val_tokens = tokenizer(X_val, padding=True, truncation=True, return_tensors="tf", max_length=64)
    test_tokens = tokenizer(X_test, padding=True, truncation=True, return_tensors="tf", max_length=64)
    
    # Extraction des tenseurs
    train_input_ids = train_tokens['input_ids']
    train_attention_mask = train_tokens['attention_mask']
    val_input_ids = val_tokens['input_ids']
    val_attention_mask = val_tokens['attention_mask']
    test_input_ids = test_tokens['input_ids']
    test_attention_mask = test_tokens['attention_mask']

    # Conversion des labels
    y_train = tf.convert_to_tensor(y_train.values, dtype=tf.int32)
    y_val = tf.convert_to_tensor(y_val.values, dtype=tf.int32)
    y_test = tf.convert_to_tensor(y_test.values, dtype=tf.int32)
    


    #-------------------------------------------------
    # 4. Création des datasets TensorFlow
    #-------------------------------------------------
    train_dataset = tf.data.Dataset.from_tensor_slices(
        ({"input_ids": train_input_ids, "attention_mask": train_attention_mask}, y_train)
    ).shuffle(len(y_train)).batch(params["batch_size"]).prefetch(tf.data.experimental.AUTOTUNE)

    validation_dataset = tf.data.Dataset.from_tensor_slices(
        ({"input_ids": val_input_ids, "attention_mask": val_attention_mask}, y_val)
    ).batch(params["batch_size"]).prefetch(tf.data.experimental.AUTOTUNE)

    test_dataset = tf.data.Dataset.from_tensor_slices(
        ({"input_ids": test_input_ids, "attention_mask": test_attention_mask}, y_test)
    ).batch(params["batch_size"]).prefetch(tf.data.experimental.AUTOTUNE)

    #-------------------------------------------------
    # 5. Définition de l'optimiseur et compilation
    #-------------------------------------------------
    optimizer = AdamWeightDecay(learning_rate=params["lr"])
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    model.compile(optimizer=optimizer, loss=loss_fn, metrics=["accuracy"])

    best_val_loss = float("inf")
    best_model_path = "best_distilbert_model"
    
    #-------------------------------------------------
    # 6. Entraînement du modèle
    #-------------------------------------------------
    full_history = {
        "loss": [],
        "val_loss": [],
        "accuracy": [],
        "val_accuracy": []
    }

    
    for epoch in range(params["epochs"]):
        history = model.fit(train_dataset, validation_data=validation_dataset, epochs=1, verbose=1)

        # Stockage des valeurs dans full_history
        full_history["loss"].append(history.history["loss"][0])
        full_history["val_loss"].append(history.history["val_loss"][0])
        full_history["accuracy"].append(history.history["accuracy"][0])
        full_history["val_accuracy"].append(history.history["val_accuracy"][0])
    
        val_loss = history.history["val_loss"][0]
        
    #-------------------------------------------------
    # 7. Log du tokenizer et du meilleur modele dans MLflow
    #-------------------------------------------------
        if val_loss < best_val_loss:
            best_val_loss = val_loss

            model.save("best_distilbert_saved_model", save_format="tf")  # Sauvegarde en format TensorFlow SavedModel
            tokenizer.save_pretrained("best_distilbert_tokenizer")  # Sauvegarde du tokenizer
            
            # Loguer le modèle SavedModel dans MLflow
            mlflow.tensorflow.log_model(model, "model")
            
            # Loguer le tokenizer comme fichier séparé dans MLflow
            mlflow.log_artifact("best_distilbert_tokenizer")  # Si nécessaire, sauvegarde le tokenizer aussi
            
    #-------------------------------------------------
    # 8. Évaluation du modèle sur test set
    #-------------------------------------------------

    y_true = []
    y_pred = []

    for batch in test_dataset:
        inputs, labels = batch
        logits = model(inputs, training=False).logits
        predictions = tf.argmax(logits, axis=-1)

        y_true.extend(labels.numpy())
        y_pred.extend(predictions.numpy())

    #-------------------------------------------------
    # 9. Résultats
    #-------------------------------------------------

    show_history(full_history)  
    plot_history(full_history, path=f"training_history.png")
    plt.close()
    
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)

    tn, fp, fn, tp = conf_matrix.ravel()
    cout_metier = fp + fn

    print("Accuracy:", round(accuracy, 3))
    print("F1 Score:", round(f1, 3))
    print("Matrice de confusion:\n", conf_matrix)
    print("Coût métier:", cout_metier)

    # Log des résultats dans MLflow
    mlflow.log_metric("Accuracy", accuracy)
    mlflow.log_metric("F1_Score", f1)
    mlflow.log_metric("Coût métier", cout_metier)
    mlflow.log_metric("FN", fn)
    mlflow.log_metric("FP", fp)


In [ ]:
#Verification des data train vs test:
print("Train class distribution:", np.bincount(y_train))
print("Test class distribution:", np.bincount(y_test))

# Conclusion

De tous les modèles testés, BERT est clariement le plus performant; c'est donc celui ci que l'on utilisera pour le déploiement en production sur Azure Web Services.